# Embedding Model with Instructions

## Datasets

In [1]:
def display_dataset_info(dataset):
    info = dataset.info
    dataset_name = info.dataset_name
    splits_info = info.splits
    features = info.features
    print(f"Dataset Name: {dataset_name}")
    print("Splits Info:")
    for split_name, split_info in splits_info.items():
        num_examples = split_info.num_examples
        print(f" - Split: {split_name}, Num Examples: {num_examples}")
    print("Features:")
    for feature_name, feature_info in features.items():
        print(f" - {feature_name}: {feature_info}")

In [2]:
from datasets import load_dataset

def get_dataset(dataset_name, train_size=0, test_size=0):  
    # Load the dataset
    dataset = load_dataset(dataset_name)
    #display_dataset_info(dataset['test'])
    
    # Access the train, test splits
    train_dataset = dataset['train']
    test_dataset = dataset['test']

    # Random sample the dataset, only use random_sample_size
    if(train_size != 0):
        train_dataset = train_dataset.shuffle(seed=42).select(range(train_size))
    if(test_size != 0):
        test_dataset = test_dataset.shuffle(seed=42).select(range(test_size))

    return train_dataset, test_dataset

## Embedding Models

In [3]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import warnings

def load_model(model_name):
    warnings.filterwarnings("ignore")
    match model_name:
        case "Bert1":
            return pipeline("feature-extraction", model="google-bert/bert-base-uncased", device=0)
        case "Bert2":
            return pipeline("feature-extraction", model="google-bert/bert-large-uncased", device=0)
        case "Instructor":
            return SentenceTransformer("hkunlp/instructor-large")
        case _:
            return None

### BERT - Pipeline

In [4]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np

def encode_Bert(model, dataset, key="text", truncation=True, padding=True, max_length=512, use_cls=True):
    data = KeyDataset(dataset, key)
    pipe = model(data, return_tensors=True, truncation=truncation, padding=padding, max_length=max_length)
    embeddings=[]
    for tensor in tqdm(pipe, desc="Encoding"): 
        if use_cls:
            # Shape [batch_size, sequence_length, hidden_size]
            embedding = tensor[0, 0, :].detach().numpy()
            print(embedding.shape)
        else:
            embedding = tensor.mean(dim=1).flatten()
        embeddings.append(embedding)
    return np.array(embeddings), np.array(dataset["label"])

### Instructor - Sentence Transformers

In [5]:
def encode_ST(model, dataset):
    embeddings = []
    texts = dataset["text"]
    instructions = dataset["instruction"]
    for text, instruction in tqdm(zip(texts, instructions),total=len(dataset), desc="Encoding"):
        embedding = model.encode([[instruction, text]])[0]
        embeddings.append(embedding)
    return np.array(embeddings), np.array(dataset["label"])

### T5 - Transformer Sentence Piece

In [6]:
from transformers import T5Tokenizer, T5Model
import torch
from tqdm import tqdm
import numpy as np

def encode_T5(dataset, model, key="text", truncation=True, padding=True, max_length=512, use_mean_pooling=True):
    # Check if CUDA is available and set device accordingly
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize the tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained(model)
    model = T5Model.from_pretrained(model).to(device)
    
    embeddings = []
    labels = []
    
    for data in tqdm(dataset, desc="Encoding text"):
        text = data[key]
        label = data.get("label", None)
        
        # Tokenize the input text
        inputs = tokenizer(text, truncation=truncation, padding=padding, max_length=max_length, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Get encoder outputs
        with torch.no_grad():
            encoder_outputs = model.encoder(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        
        # Pooling to get a single vector for each input
        if use_mean_pooling:
            attention_mask = inputs['attention_mask']
            last_hidden_state = encoder_outputs.last_hidden_state
            mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
            sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
            embedding = sum_embeddings / sum_mask
        else:
            embedding = encoder_outputs.last_hidden_state[:, 0, :]
        
        embeddings.append(embedding.cpu().numpy().flatten())  # Flatten the embeddings
        if label is not None:
            labels.append(label)
    
    return np.array(embeddings), np.array(labels)

### GPT2 - Sentence Transformer

In [19]:
import torch
from transformers import GPT2Tokenizer, GPT2Model
from torch.utils.data import DataLoader

def encode_GPT(dataset, model_name="gpt2", key="text", truncation=True, padding=True, max_length=512, use_mean_pooling=True, batch_size=16):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    
    # Set pad_token to eos_token
    tokenizer.pad_token = tokenizer.eos_token

    model = GPT2Model.from_pretrained(model_name)
    model.eval()

    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    def embed_texts(batch):
        inputs = tokenizer(batch, truncation=truncation, padding=padding, max_length=max_length, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        if use_mean_pooling:
            embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling
        else:
            embeddings = outputs.last_hidden_state[:, 0, :]  # Use CLS token (if applicable)
        return embeddings.cpu().numpy()

    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    all_embeddings = []
    all_labels = []

    for batch in dataloader:
        texts = batch[key]
        embeddings = embed_texts(texts)
        all_embeddings.extend(embeddings)
        all_labels.extend(batch['label'])
    
    return all_embeddings, all_labels

## Augment Instruction

In [8]:
def mapper_affixes(example, prefix, suffix):
    example['text'] = prefix + example['text'] + suffix
    return example

def mapper_instruct(example, instruction):
    example['instruction'] = instruction
    return example

def augment_dataset_Affix(dataset, prefix, suffix):
    augmented_dataset = dataset.map(lambda x: mapper_affixes(x, prefix, suffix))
    return augmented_dataset

def augment_dataset_Inst(dataset, instruction):
    augmented_dataset = dataset.map(lambda x: mapper_instruct(x, instruction))
    return augmented_dataset

## Evaluation

In [9]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

def evaluate(method, train_embeddings, test_embeddings, train_labels, test_labels):
    if method == "SVM":
        model = SVC(kernel='linear')
        
    elif method == "MLP":
        model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                          solver='sgd', verbose=1, random_state=1,
                          learning_rate_init=.1)

    model.fit(train_embeddings, train_labels)
    predicted_labels = model.predict(test_embeddings)
    print("Report on " + method + ": ")
    print(classification_report(y_true = test_labels, y_pred = predicted_labels, digits=4))

## EmbedFlow

In [10]:
def EmbedFlow_Bert_1(dataset_name, train_size, test_size, evaluator, prefix, suffix):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)

    # Load Model
    model = pipeline("feature-extraction", model="google-bert/bert-base-uncased", device=0)
    
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Bert(model, train_dataset, use_cls=False)
    test_embeddings, test_labels = encode_Bert(model, test_dataset, use_cls=False)

    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

In [11]:
def EmbedFlow_Bert_2(dataset_name, train_size, test_size, evaluator, prefix, suffix):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)

    # Load Model
    model = pipeline("feature-extraction", model="google-bert/bert-large-uncased", device=0)
    
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Bert(model, train_dataset, use_cls=False)
    test_embeddings, test_labels = encode_Bert(model, test_dataset, use_cls=False)

    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

In [12]:
def EmbedFlow_Instructor(dataset_name, train_size, test_size, evaluator, instruction):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)

    # Load Model
    model = SentenceTransformer("hkunlp/instructor-large")
    
    # Add Instruction
    train_dataset = augment_dataset_Inst(train_dataset, instruction)
    test_dataset = augment_dataset_Inst(test_dataset, instruction)

    # Embed Dataset
    train_embeddings, train_labels = encode_ST(model, train_dataset)
    test_embeddings, test_labels = encode_ST(model, test_dataset)
    
    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

In [13]:
def EmbedFlow_T5(dataset_name, train_size, test_size, evaluator, prefix, suffix):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)
  
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_T5(train_dataset, model="t5-base")
    test_embeddings, test_labels = encode_T5(test_dataset, model="t5-base")
    
    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

In [17]:
def EmbedFlow_GPT(dataset_name, train_size, test_size, evaluator, prefix, suffix):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)
  
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_GPT(train_dataset, model_name="gpt2")
    test_embeddings, test_labels = encode_GPT(test_dataset, model_name="gpt2")
    
    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

## Testing
[Google Sheet](https://docs.google.com/spreadsheets/d/1iBDq7C59G6olf_of_sTF5oCY3Itj6_kImzeUl3XMpd8/edit#gid=1587051763)

In [21]:
import warnings
warnings.filterwarnings("ignore")

datasets = ['stanfordnlp/imdb', 
            'yelp_review_full',
            'Voice49/arXiv-Abstract-Label-20k']

evaluator = ['SVM', 'MLP']

prefix = ['',
          'Movie Review: ', 
          'Restaurant Review: ', 
          'Sentiment Analysis: ', 
          'User Feedback: ', 
          'Customer Experience: ',
          'Product Review: ',
          'Service Feedback: ',
          'Experience at: ',
          'Abstract: ',
          'Research Paper Abstract:',
          'Paper Summary: ']
suffix = ['','']

train_size = 1000
test_size  = 1000

#EmbedFlow_Bert_1(datasets[2], train_size, test_size, evaluator[0], prefix[0], suffix[0])
EmbedFlow_GPT(datasets[2], train_size, test_size, evaluator[0], prefix[0], suffix[0])

Processing dataset: stanfordnlp/imdb, instruction: ''


c:\Users\Voice\anaconda3\envs\embed\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Report on SVM: 
              precision    recall  f1-score   support

        cs.*       0.50      0.50      0.50         2
      econ.*       0.00      0.00      0.00         1
      math.*       0.50      1.00      0.67         1
   physics.*       0.00      0.00      0.00         1
     q-bio.*       0.00      0.00      0.00         1
     q-fin.*       1.00      1.00      1.00         1
      stat.*       0.40      0.67      0.50         3

    accuracy                           0.50        10
   macro avg       0.34      0.45      0.38        10
weighted avg       0.37      0.50      0.42        10



c:\Users\xsyyy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\xsyyy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\xsyyy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,